In [484]:
from gurobipy import*
m = Model("research")

# Parameter

In [485]:

#file = 'benchmark_new_1'
path = 'AUO/testdata_0805_final_2/benchmark_new/%s.txt' % (file) 

with open(path) as a:
    col = len(a.readlines())
a.close()
l = [0]*col
c = 0
with open(path) as f:
    for line in f.readlines():
        l[c] = line.split(' ')
        c = c + 1
f.close()        
ma = int(l[0][0]) # numbers of machines
h = int(l[0][1]) # #of machines that can be maintained at the same time
b = int(l[0][2]) # maintenance time

n = [0]
for i in range(ma):
    n.append(int(l[1][i]))

rB = [0] # defect rate "before" maintenance
rB.append(float(l[2][1]))
rB.append(float(l[2][2]))
rA = [0] # defect rate "after" maintenance
rA.append(float(l[3][1]))
rA.append(float(l[3][2]))

s = [[0] for i in range(3)] # starting time
p = [[0] for i in range(3)] # processing time
d = [[0] for i in range(3)] # due date

for count in range(1,ma+1):
    for i in range(4,len(l)):
        if int(l[i][0]) == count:
            for j in range(len(l[i][0])):
                s[count].append(int(l[i][2]))
                p[count].append(int(l[i][3]))
                d[count].append(int(l[i][4]))            

T = 500

# Decision Variables

In [486]:
X = {}
Y = {}
W = {}
Z = {}
A = {}

for i in range(1,ma+1): 
    for j in range(n[i]+1):
        X[i,j] = m.addVar(vtype="B",name = "X(%s,%s)"%(i,j))
for i in range(1,ma+1):
    for j in range(n[i]+1):
        Y[i,j] = m.addVar(vtype="B",name = "Y(%s,%s)"%(i,j))
for i in range(1,ma+1):
    for j in range(n[i]+1):
        W[i,j] = m.addVar(vtype="I",name = "W(%s,%s)"%(i,j))
for i in range(1,ma+1):
    for j in range(n[i]+1):
        Z[i,j] = m.addVar(vtype="I",name = "Z(%s,%s)"%(i,j))
for i in range(1,ma+1):
    for j in range(n[i]+1):
        for t in range(T):
            A[i,j,t] = m.addVar(vtype="B",name = "A(%s,%s,%s)"%(i,j,t))
m.update()

# Constraints

In [487]:
m.addConstr(X[1,0]==0)
m.addConstr(X[2,0]==0)

for (i,j) in X:
    if j != 0:
        m.addConstr(Y[i,j] >= X[i,j])
    
for i in range(1, ma + 1): # There is at most one maintenance on the machine i
    m.addConstr(quicksum(X[i,j] for j in range(n[i] + 1)) <= 1)
    
for (i,j) in Y:
    m.addConstr(Y[i,j] <= quicksum(X[i,k] for k in range(1,j+1)))

for (i,j) in Z:
    m.addConstr(Z[i,j] >= s[i][j] + p[i][j] + W[i,j] - d[i][j])

for (i,j) in W:
    m.addConstr(W[i,j] >= b * quicksum(X[i,k] for k in range(1,j+1)) - quicksum(Y[i,k] * (s[i][k]-s[i][k-1] - p[i][k-1]) for k in range(1,j+1)))
                
for t in range(T):
    m.addConstr(quicksum(quicksum(A[i,j,t] * X[i,j] for j in range(n[i]+1)) for i in range(1,ma+1)) <= h)

for (i,j) in X:
    for t in range(T):
        if t in range(s[i][j] + p[i][j] + 1, s[i][j] + p[i][j]+ b +1):
            m.addConstr(A[i,j,t] == 1)
        else:
            m.addConstr(A[i,j,t] == 0)

m.update()    

# Objective

In [488]:
m.setObjective(100*quicksum(p[i][j] * ((rA[i] * Y[i,j]) + rB[i] * (1 - Y[i,j])) for (i,j) in Y)
               + quicksum(Z[i,j] for (i,j) in Z),GRB.MINIMIZE)
m.optimize()
m.write('model1.lp')
m.write('model1.sol')

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8570 rows, 8568 columns and 8789 nonzeros
Model fingerprint: 0xe5d578f2
Model has 500 quadratic constraints
Variable types: 0 continuous, 8568 integer (8534 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 9e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
  QRHS range       [1e+00, 1e+00]
Presolve removed 8532 rows and 8534 columns
Presolve time: 0.01s
Presolved: 38 rows, 34 columns, 181 nonzeros
Variable types: 0 continuous, 34 integer (24 binary)
Found heuristic solution: objective 10029.000000
Found heuristic solution: objective 10010.000000
Found heuristic solution: objective 9782.0000000

Root relaxation: cutoff, 8 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  O

# Output

In [489]:
print("Optimal value:", m.ObjVal)
#m.printAttr('X')

Optimal value: 9781.999999999998


In [490]:
with open("model1.sol",'r') as e:
    col = len(e.readlines())
e.close()

ll = [0]*col
c = 0
with open("model1.sol",'r') as v:
    for line in v.readlines():
        ll[c] = line.split(' ')
        c = c + 1
v.close()
    
with open("1",'w') as vv:
    vv.write('%s,%s\n' % (ma,T))
    for i in range(1,ma+1):
        vv.write('%s '%n[i])
    vv.write('\n')
    for i in range(2,col):
        vv.write('%s,%s' % (ll[i][0],ll[i][1]))
    
vv.close()

data = data2 = ""
  
# Reading data from file1
with open(path) as o:
    data2 = o.read()
o.close()
# Reading data from file2
with open("1") as z:
    data = z.read()
z.close()

data += data2
  
with open ("%s_sol.txt"%file, 'w') as fp:
    fp.write(data)
fp.close()
